In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import math

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import numpy as np
from numpy import random

In [2]:
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe):
        self.X = torch.tensor(dataframe.iloc[:, :3].values, dtype=torch.float32)  # Input: [Batch, 3]
        self.Y = torch.tensor(dataframe.iloc[:, 3:].values, dtype=torch.float32).unsqueeze(-1) # Output: [Batch, 255, 1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [3]:
# Positional Encoding for Time Steps
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)  # Add batch dimension

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :].to(x.device)

# Transformer Model
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward):
        super(TransformerTimeSeriesModel, self).__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
 
        # Input Encoder (maps input to d_model size)
        self.encoder = nn.Linear(input_dim, d_model)  # (Batch, 3) -> (Batch, d_model)
        
        # Project input to match the sequence length
        self.expand_input = nn.Linear(d_model, seq_length * d_model)  # (Batch, d_model) -> (Batch, seq_length * d_model)
        
        # Target embedding for decoder input
        self.target_embedding = nn.Linear(1, d_model)  # New embedding layer for target sequence
  
        # Positional Encoding for Time Steps
        self.pos_encoder = PositionalEncoding(d_model, seq_length)
        
        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        # Final Output Layer
        self.output_layer = nn.Linear(d_model, output_dim)  # (Batch, 255, d_model) -> (Batch, 255, 1)

    def forward(self, x, target_seq):
        # x: Input features [Batch, 3]
        # target_seq: Target sequence for teacher forcing [Batch, 255, 1]
        
        # Encode input features
        encoded_input = self.encoder(x)  # [Batch, d_model]
        
        # Expand input to match sequence length
        expanded_input = self.expand_input(encoded_input)  # [Batch, seq_length * d_model]
        expanded_input = expanded_input.view(-1, self.seq_length, self.d_model)  # Reshape to [Batch, 255, d_model]
        
        # Add Positional Encoding
        expanded_input = self.pos_encoder(expanded_input)
        
        # Process the target sequence through the same encoding pipeline
  #      target_embeddings = self.encoder(target_seq)
  #      target_embeddings = nn.Linear(1, d_model)(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.target_embedding(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.pos_encoder(target_embeddings)
        
        # Decode sequence
        output = self.transformer_decoder(
            tgt=target_embeddings, memory=expanded_input
        )  # Output shape: [Batch, 255, d_model]
        
        # Map to output dimensions
        predictions = self.output_layer(output)  # [Batch, 255, 1]
        return predictions

In [4]:
def train_model(model, dataloader, optimizer, loss_fn, num_epochs, device):
    loss_list = list()
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch in dataloader:
            x, y = batch  # x: [Batch, N], y: [Batch, T]
            x, y = x.to(device), y.to(device)
            
            # Prepare target for teacher forcing
            target_seq = y 
            #target_seq = y[:, :-1]  # All except last time step
            #actual = y[:, 1:]       # All except first time step
            
            # Forward pass
            output = model(x, target_seq)
            loss = loss_fn(output, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_list.append(loss.item())
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
    return loss_list

In [15]:
# Load the CSV file
data_input = pd.read_csv("~/Desktop/TS-Clustering/SimData/epsteinCV_inputs.csv", sep=" ", header=None)
data_output = pd.read_csv("~/Desktop/TS-Clustering/SimData/epsteinCV_outputs_active.csv", header=None)
data = pd.concat([data_input, data_output], axis=1)

scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data)

# Split the data into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the validation set to a new CSV file
valid_data.to_csv("validation_set.csv", index=False)

In [17]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
3547,0.553148,0.109513,0.033030,0.0,0.407305,0.478291,0.413508,0.341144,0.263267,0.197795,...,0.008959,0.002757,0.001378,0.002757,0.004824,0.006203,0.004135,0.006892,0.001378,0.007581
34861,0.965540,0.031016,0.216775,0.0,0.747760,0.853205,0.837354,0.824259,0.807030,0.795314,...,0.583046,0.579600,0.591316,0.595451,0.594762,0.587870,0.593384,0.594073,0.589249,0.593384
18274,0.949296,0.038281,0.495498,0.0,0.604411,0.773260,0.760855,0.744314,0.724328,0.711923,...,0.493453,0.490696,0.494831,0.495520,0.492764,0.493453,0.492074,0.504480,0.500345,0.492764
33070,0.720729,0.248677,0.152559,0.0,0.239835,0.177808,0.038594,0.002068,0.000000,0.000000,...,0.002068,0.000689,0.001378,0.002068,0.004135,0.000689,0.000689,0.002757,0.002068,0.001378
29702,0.238838,0.756370,0.037248,0.0,0.019297,0.000000,0.000000,0.000000,0.000689,0.000000,...,0.000689,0.000000,0.000000,0.000000,0.000000,0.001378,0.000000,0.000689,0.000689,0.000000


In [19]:
# Load your DataFrame (assuming it's named `df`)
dataset = TimeSeriesDataset(train_data)

In [21]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
# Model parameters
input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)


In [25]:
# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 40  # Adjust based on dataset size and performance
train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

Epoch 1/40, Loss: 0.00039567871135659516
Epoch 2/40, Loss: 0.00016801682068035007
Epoch 3/40, Loss: 8.223417535191402e-05
Epoch 4/40, Loss: 0.00017714404384605587
Epoch 5/40, Loss: 0.00023919713567011058
Epoch 6/40, Loss: 3.275905328337103e-05
Epoch 7/40, Loss: 0.00014134531375020742
Epoch 8/40, Loss: 2.9211678338469937e-05
Epoch 9/40, Loss: 3.313839988550171e-05
Epoch 10/40, Loss: 1.0398732229077723e-05
Epoch 11/40, Loss: 7.4640652201196644e-06
Epoch 12/40, Loss: 7.13620056558284e-06
Epoch 13/40, Loss: 6.548744295287179e-06
Epoch 14/40, Loss: 1.201140275952639e-05
Epoch 15/40, Loss: 0.022827692329883575
Epoch 16/40, Loss: 0.00010908328840741888
Epoch 17/40, Loss: 3.435953112784773e-05
Epoch 18/40, Loss: 5.849183708050987e-06
Epoch 19/40, Loss: 7.250102498801425e-06
Epoch 20/40, Loss: 1.856780545494985e-05
Epoch 21/40, Loss: 0.00037825110484845936
Epoch 22/40, Loss: 0.00010496236791368574
Epoch 23/40, Loss: 0.013372762128710747
Epoch 24/40, Loss: 0.044779662042856216
Epoch 25/40, Loss:

In [27]:
torch.save(model.state_dict(), "transformer_adam_lr001_epstein.pth")

In [7]:
model = TransformerTimeSeriesModel(
    input_dim=3, output_dim=1, seq_length=252, 
    d_model=128, nhead=4, num_layers=2, dim_feedforward=512
)

# Load the saved weights
model.load_state_dict(torch.load("C:/Users/met48/Desktop/ABM-Surrogate/transformer_adam_lr001_epstein.pth"))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/met48/Desktop/ABM-Surrogate/transformer_adam_lr001_epstein.pth'

In [70]:
#model.load_state_dict(torch.load("transformer_adam_lr001_epstein.pth", map_location=torch.device("cpu")))
with open('epsteinCV_inputs_surrogate.csv', 'w', newline='') as epsteinCV_inputs, \
    open('epsteinCV_outputs_active_surrogate.csv', 'w', newline='') as epsteinCV_outputs_active:
    model.eval()  # Set model to evaluation mode
    batch_size = 1  # Inference on a single sample
    seq_length = 252
    for i in np.arange(10000):
        cit_dens = random.uniform(0.5, 1)
        cop_dens = random.uniform(0, 1-cit_dens)
        leg = random.uniform(0, 1)
        input_features = torch.tensor([[cit_dens, cop_dens, leg]], dtype=torch.float32)  # Shape: [1, 3]
        
        # Initialize target sequence with zeros for inference
        target_seq = torch.zeros((batch_size, seq_length, 1), dtype=torch.float32)  # Shape: [1, 252, 1]
        with torch.no_grad():
            predictions = model(input_features, target_seq)  # Output shape: [1, 252, 1]
        
        # Convert to NumPy array for easy viewing
        predicted_values = predictions.squeeze().numpy()  # Shape: [255]
        print(*list([cit_dens, cop_dens, leg]), file=epsteinCV_inputs)
        epsteinCV_outputs_active.write(" ".join(f"{value:.6f}" for value in predicted_values.flatten()))
        #print(predicted_values.flatten().tolist(), file=epsteinCV_outputs_active, sep=",")
epsteinCV_inputs.close()
epsteinCV_outputs_active.close()

In [12]:
import tslearn
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from yellowbrick.cluster import KElbowVisualizer
import pandas as pd
import numpy as np

In [ ]:
seed=1
train_inputs = pd.read_csv('epsteinCV_outputs_active_surrogate.csv', sep=' ')
X_train = to_time_series_dataset(train_inputs)
X_train_flat = [xi.flatten() for xi in X_train]

km = TimeSeriesKMeans(verbose=True, random_state=seed)
visualizer = KElbowVisualizer(km, k=(2,16), locate_elbow=False)
 
visualizer.fit(np.array(X_train_flat))        
visualizer.show()   

In [ ]:
def ts_cluster_visualization(y_pred, df, n_clusters, plot_title):
    ts_size = df.shape[1]
    ts_max = df.max()
    plt.figure()
    for cluster in range(n_clusters):
        plt.subplot(4, math.ceil(n_clusters/4), cluster+1)
        for ts in df[y_pred == cluster]:
            plt.plot(ts.ravel(), "k-", alpha=.2)
        plt.plot(np.mean(df[y_pred == cluster], axis=0), "r-")
        plt.xlim(0, ts_size)
        plt.ylim(0, ts_max)
        plt.text(0.55, 0.35,'Cluster %d' % (cluster),
                 transform=plt.gca().transAxes)
        if cluster == 1:
            plt.title(plot_title)      
    plt.tight_layout()
    plt.show()